<div class="alert alert-block alert-success">
    <b><center>Pytorch LIBRARY</center></b>
    <b><center>MNIST ANN</center></b>
</div>

# Configure Learning Environment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import torch
from torch import nn
from torchvision import datasets, transforms

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Prepare Data

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])
res_train = datasets.MNIST("./datas", train=True, transform=transform, download=True)

100%|██████████| 9.91M/9.91M [00:04<00:00, 2.46MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 159kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.41MB/s]


In [5]:
load_train = torch.utils.data.DataLoader(res_train, batch_size=100, shuffle=True)

In [6]:
iter_train = iter(load_train)
images, labels = iter_train.next()
images.shape, labels.shape

AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'

In [ ]:
def im_convert(tensor):
    image = tensor.clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
    image = image.clip(0, 255)
    return image

In [ ]:
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    plt.imshow(im_convert(images[idx]))
    ax.set_title(labels[idx].item())

# Generate Model

In [ ]:
class Classifier(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
        super().__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.activate1 = nn.ReLU()
        self.linear2 = nn.Linear(H1, H2)
        self.activate2 = nn.ReLU()
        self.linear3 = nn.Linear(H2, D_out)
    
    def forward(self, x):
        x = self.activate1(self.linear1(x))
        x = self.activate2(self.linear2(x))
        x = self.linear3(x)
        return x

In [ ]:
model = Classifier(784, 125, 65, 10).to(device)
model

In [ ]:
fn_loss = nn.CrossEntropyLoss()
fn_optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training

In [ ]:
epochs = 20
amount_batch = len(load_train)

In [ ]:
history = []
for epoch in range(epochs):
    loss_learns = .0
    accu_learns = .0
    
    for datas, labels in load_train:
        # set device
        datas = datas.to(device)
        labels = labels.to(device)
        
        # predict & calculate a loss
        inputs = datas.view(datas.shape[0], -1)
        preds = model(inputs)
        loss = fn_loss(preds, labels)
        
        # backpropagation
        fn_optimizer.zero_grad()
        loss.backward()
        fn_optimizer.step()
        
        loss_learns += loss.item()
        _, pred_label = torch.max(preds, 1)
        accu_learns += torch.sum(pred_label == labels.data).item() / len(pred_label) * 100
        
    loss_learn = loss_learns / amount_batch
    accu_learn = accu_learns / amount_batch
    history.append({"loss": loss_learn, "accuracy": accu_learn})
    
    print(f"{epoch+1:02d} step -> loss: {loss_learn:.4f}, accuracy: {accu_learn:.2f} %")

In [ ]:
df_history = pd.DataFrame.from_records(history)
df_history.plot(subplots=True, layout=(1, 2), figsize=(15, 5))